In [1]:
import tensorflow as tf
import numpy as np

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values to [0,1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten images: (28, 28) → (784,)
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Add bias term: Append a column of ones (Now shape: (N, 785))
x_train_bias = np.hstack([x_train_flat[:2000], np.ones((2000, 1))])
x_test_bias = np.hstack([x_test_flat[:500], np.ones((500, 1))])

# Select corresponding labels & reshape to column vectors
y_train_sampled = y_train[:2000].reshape(-1, 1)
y_test_sampled = y_test[:500].reshape(-1, 1)

# Print final dataset shapes
print(f"Training set: {x_train_bias.shape}, Labels: {y_train_sampled.shape}")
print(f"Test set: {x_test_bias.shape}, Labels: {y_test_sampled.shape}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training set: (2000, 785), Labels: (2000, 1)
Test set: (500, 785), Labels: (500, 1)


In [2]:
W = np.random.randn(785, 10) * 0.001
num_classes = W.shape[1]

In [3]:
def softmax_vectorized(X, y, W, reg):
  num_train = X.shape[0]
  num_classes = W.shape[1]

  scores = np.dot(X, W)
  scores -= np.max(scores, axis=1, keepdims=True)

  exp_scores = np.exp(scores)
  softmax_probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

  correct_class_probs = softmax_probs[np.arange(num_train), y.flatten()]
  loss = - np.sum(np.log(correct_class_probs)) / num_train
  loss += reg * np.sum(W * W)

  softmax_probs[np.arange(num_train), y.flatten()] -= 1
  dW = np.dot(X.T, softmax_probs) / num_train
  dW += 2 * reg * W

  return loss, dW

In [7]:
def softmax_train_vectorized(X, y, W, reg, alpha=0.01, iters=100):
  num_train = X.shape[0]
  for i in range(iters):
    loss, dW = softmax_vectorized(X, y, W, reg)
    W -= alpha * dW

    if i % 10 == 0:
      print(f"Epoch {i}: Loss = {float(loss):.4f}")
  return W

In [8]:
W = softmax_train_vectorized(x_train_bias, y_train_sampled, W, reg=0.0001, alpha=0.01, iters=100)

Epoch 0: Loss = 2.3032
Epoch 10: Loss = 2.1968
Epoch 20: Loss = 2.0985
Epoch 30: Loss = 2.0074
Epoch 40: Loss = 1.9228
Epoch 50: Loss = 1.8444
Epoch 60: Loss = 1.7717
Epoch 70: Loss = 1.7044
Epoch 80: Loss = 1.6421
Epoch 90: Loss = 1.5843


In [9]:
y_pred = np.dot(x_test_bias, W)
y_pred = np.argmax(y_pred, axis=1)
accuracy = np.mean(y_pred == y_test_sampled)

In [10]:
accuracy

np.float64(0.102644)